In [36]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt

In [37]:
# !pip install --upgrade pythainlp
# !pip install pyLDAvis

In [38]:
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings


In [39]:
df = pd.read_csv("sentimentMacbookAirM1.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  81 non-null     object
dtypes: object(1)
memory usage: 776.0+ bytes


In [40]:
df = df.astype(str)
df

,message
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก
...,...
76,ขออนุญาตสอบถามหน่อยนะคะ คือเราซื้อ macbook pro...
77,อยากสอบถามทุกคนหน่อยค่ะ ว่า apple care+ ควรซื้...
78,ตั้งแต่เราซื้อผลิตภัณฑ์แอปเปิ้ลมาลังเลตลอดว่าจ...
79,มันก็ออกมาเรื่อยทุกปีแหละครับ ดูการใช้งานและปร...


# Tokenize word

In [41]:
stop_word = list(pythainlp.corpus.thai_stopwords())
rm_word = [ ' ','    ','\n', '(', ')', '?', '+', 'ผม', ':', 'ตัว', 'เรื่อง', 'ส่วนตัว', 'คับ', '--', '//', 'แน่นอน', 'สอย', 'ซื้อ', 'เวลา', '' ]
screenword = stop_word + rm_word

def tokenize_n_space(sentence):
    merged = ''
    words = pythainlp.word_tokenize( str(sentence), engine = 'newmm' )
    for i in words :
        if i not in screenword :
            merged = merged + ',' + i
    return merged[1:]

In [42]:
df['tokenized'] = df['message'].apply(lambda x: tokenize_n_space(x))
df

,message,tokenized
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...,"คอนเฟิร์ม,เสียง,Macbook,air,m,1,ลื่นไหล,แบต,อึ..."
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...,"สอบถาม,กะ,mac,air,m,1,ทำงาน,ตัดต่อ,กราฟิก,phot..."
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...,"M,1,ตัดต่อ,ระดับ,เทพ,ๆๆๆ,โปรแกรม,พัดลม"
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...,"เล่น,เกม,โน๊ตบุ๊ค,ดี,โลก,ราคา,28000,ทำได้,ประส..."
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก,"จุดเด่น,เสียง,ลำโพง,ดีมาก"
...,...,...
76,ขออนุญาตสอบถามหน่อยนะคะ คือเราซื้อ macbook pro...,"ขออนุญาต,สอบถาม,นะคะ,macbook,pro,14,”,m,1,pro,..."
77,อยากสอบถามทุกคนหน่อยค่ะ ว่า apple care+ ควรซื้...,"สอบถาม,apple,care,เครื่อง,icare,ต่างกัน, ,กลัว..."
78,ตั้งแต่เราซื้อผลิตภัณฑ์แอปเปิ้ลมาลังเลตลอดว่าจ...,"ผลิตภัณฑ์,แอปเปิ้ล,ลังเล,บ้าน,apple,ประกัน,ตอน..."
79,มันก็ออกมาเรื่อยทุกปีแหละครับ ดูการใช้งานและปร...,"ออกมา,ทุกปี,ดู,การใช้งาน,ประโยชน์,ที่จะ,คน,รอ,..."


# Creating Dictionary

In [43]:
doc = df['tokenized'].to_list()
texts = [ [i for i in dc.split(',')] for dc in doc ]

dictionary = gensim.corpora.Dictionary(texts)

print(dictionary.token2id.keys())

dict_keys(['1', 'Macbook', 'Performance', 'Windows', 'air', 'drop', 'm', 'คอนเฟิร์ม', 'ลืม', 'ลื่นไหล', 'สักพัก', 'อึด', 'เสียง', 'แบต', 'illustrator', 'mac', 'photoshop', 'กราฟิก', 'กะ', 'ตัดต่อ', 'ทำงาน', 'น้อง', 'สอบถาม', 'ไหว', 'M', 'พัดลม', 'ระดับ', 'เทพ', 'โปรแกรม', 'ๆๆๆ', '28000', 'การใช้งาน', 'ดี', 'ต่อให้', 'ทำ', 'ทำได้', 'ประสบการณ์', 'ราคา', 'ห่า', 'เกม', 'เล่น', 'โน๊ตบุ๊ค', 'โลก', 'จุดเด่น', 'ดีมาก', 'ลำโพง', '7', 'CTW', 'geniusbar', 'studio', 'ขาย', 'คนอื่น', 'คอม', 'จี', 'ดีกว่า', 'ดู', 'ที้ง', 'นิดนึง', 'บริการ', 'พนักงาน', 'ระบาย', 'รุ', 'ร้าน', 'สอง', 'สัก', 'สาขา', 'ห่วย', 'เข้าไป', 'เจอ', 'เนาะ', 'เนียส', 'แย่', 'แอปเปิ้ล', 'ใช้งาน', '2013', '2020', '2023', '34', '900', 'Air', 'คน', 'ค่ำ', 'งานเอกสาร', 'ตอน', 'ต้นปี', 'ประชุม', 'ประทับใจ', 'ปี', 'มีปัญหา', 'ยัน', 'รุ่น', 'หาย', 'เก่า', 'เครื่อง', 'เดิม', 'เปิดตัว', 'เลือก', 'แชร์', 'แต่เช้า', 'ไว', '/', '4', 'adobe', 'apple', 'exel', 'premia', 'word', 'การตัดต่อ', 'ขั้นพื้นฐาน', 'ความแตกต่าง', 'ค่', 'ง', 'จ่าย', 'ดร'

In [44]:
gensim_corpus = [ dictionary.doc2bow(tx, allow_update = True) for tx in texts ]
word_frq = [ [ (dictionary[ides], frq) for ides, frq in coup ] for coup in gensim_corpus ]

word_frq

[[('1', 1),
  ('Macbook', 1),
  ('Performance', 1),
  ('Windows', 1),
  ('air', 1),
  ('drop', 1),
  ('m', 1),
  ('คอนเฟิร์ม', 1),
  ('ลืม', 1),
  ('ลื่นไหล', 1),
  ('สักพัก', 1),
  ('อึด', 1),
  ('เสียง', 1),
  ('แบต', 1)],
 [('1', 1),
  ('air', 1),
  ('m', 1),
  ('illustrator', 1),
  ('mac', 1),
  ('photoshop', 1),
  ('กราฟิก', 1),
  ('กะ', 1),
  ('ตัดต่อ', 1),
  ('ทำงาน', 1),
  ('น้อง', 1),
  ('สอบถาม', 1),
  ('ไหว', 1)],
 [('1', 1),
  ('ตัดต่อ', 1),
  ('M', 1),
  ('พัดลม', 1),
  ('ระดับ', 1),
  ('เทพ', 1),
  ('โปรแกรม', 1),
  ('ๆๆๆ', 1)],
 [('Windows', 1),
  ('28000', 2),
  ('การใช้งาน', 1),
  ('ดี', 1),
  ('ต่อให้', 1),
  ('ทำ', 1),
  ('ทำได้', 1),
  ('ประสบการณ์', 1),
  ('ราคา', 3),
  ('ห่า', 1),
  ('เกม', 1),
  ('เล่น', 1),
  ('โน๊ตบุ๊ค', 2),
  ('โลก', 1)],
 [('เสียง', 1), ('จุดเด่น', 1), ('ดีมาก', 1), ('ลำโพง', 1)],
 [('ประสบการณ์', 1),
  ('7', 1),
  ('CTW', 1),
  ('geniusbar', 1),
  ('studio', 1),
  ('ขาย', 1),
  ('คนอื่น', 1),
  ('คอม', 1),
  ('จี', 1),
  ('ดีกว่า', 1),
  ('ด

# Topic Modeling

In [45]:
num_topic = 30
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_number = 1 # more number, more compute time spending

# Make index to word dictionary
temp = dictionary[0]
id2word = dictionary.id2token

%time
model = gensim.models.LdaModel( corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, alpha='auto', eta='auto'
                               , iterations=iterations, num_topics=num_topic, passes=passes, eval_every=eval_number )

Wall time: 0 ns


In [46]:
pyLDAvis.gensim.prepare( model, gensim_corpus, dictionary )

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.143383 -0.093544       1        1  9.103966
25    -0.062893 -0.051028       2        1  8.032446
3     -0.076425  0.072532       3        1  6.056747
9     -0.101701  0.112846       4        1  5.642195
22     0.103641  0.187927       5        1  5.393750
2     -0.092950  0.023324       6        1  4.943808
15     0.118165 -0.114528       7        1  4.575023
0      0.061427 -0.019713       8        1  4.573585
19     0.010210 -0.073446       9        1  4.520922
6     -0.038899  0.123550      10        1  3.923585
20     0.026309 -0.079565      11        1  3.865289
1     -0.069002  0.023269      12        1  3.812816
24     0.066353  0.090634      13        1  3.755933
14    -0.126526  0.013397      14        1  3.731796
8     -0.024452 -0.028480      15        1  3.125015
7      0.005329 -0.065901      16        1  2.992234
5     -0.014660 -0.010178      17        1  2.918850
23    -0.009050 -0.048372      18        1  2.837699
18     0.013984 -0.021224      19        1  2.521927
21    -0.057198 -0.030608      20        1  2.393770
17     0.079231 -0.033305      21        1  2.202627
12     0.028347  0.041221      22        1  1.860554
4      0.104615 -0.032693      23        1  1.833794
13     0.056202  0.007595      24        1  1.464625
27     0.075093  0.061386      25        1  1.200706
29     0.013091 -0.002215      26        1  0.832806
28    -0.010128 -0.015613      27        1  0.728169
26     0.016650 -0.023917      28        1  0.569366
10     0.010097 -0.035888      29        1  0.517459
11     0.038524  0.022538      30        1  0.068539, topic_info=            Term       Freq      Total Category  logprob  loglift
0              1  32.000000  32.000000  Default  30.0000  30.0000
24             M  18.000000  18.000000  Default  29.0000  29.0000
6              m  20.000000  20.000000  Default  28.0000  28.0000
37          ราคา  13.000000  13.000000  Default  27.0000  27.0000
73        ใช้งาน  12.000000  12.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
489  performance   0.001491   2.034341  Topic30  -6.7713   0.0671
433     นั้นแหละ   0.001491   2.148499  Topic30  -6.7713   0.0125
495          นึง   0.001491   2.167375  Topic30  -6.7713   0.0037
32            ดี   0.001491  11.718199  Topic30  -6.7714  -1.6840
79           Air   0.001491   7.562865  Topic30  -6.7714  -1.2461

[1411 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
169       2  0.202719       
169       3  0.337865       
169       4  0.202719       
169       9  0.202719       
818      24  0.624239       
...     ...       ...    ...
306      17  0.275692  ไอแพด
266      17  0.792103  ไอโฟน
825      24  0.624239     ๆ?
29       20  0.822107    ๆๆๆ
833      20  0.822107      ”

[1341 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 26, 4, 10, 23, 3, 16, 1, 20, 7, 21, 2, 25, 15, 9, 8, 6, 24, 19, 22, 18, 13, 5, 14, 28, 30, 29, 27, 11, 12])

In [47]:
model.show_topic(1)

[('1', 0.029697714),
 ('เจอ', 0.019909294),
 ('เหมือนกัน', 0.019909156),
 ('บริการ', 0.019907204),
 ('ดู', 0.019907204),
 ('สอง', 0.019907204),
 ('พนักงาน', 0.019907204),
 ('7', 0.019907203),
 ('m', 0.019907199),
 ('air', 0.019907199)]

In [48]:
df['topics'] = df['tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [49]:
df

,message,tokenized,topics,score
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...,"คอนเฟิร์ม,เสียง,Macbook,air,m,1,ลื่นไหล,แบต,อึ...",23,0.959221
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...,"สอบถาม,กะ,mac,air,m,1,ทำงาน,ตัดต่อ,กราฟิก,phot...",9,0.956464
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...,"M,1,ตัดต่อ,ระดับ,เทพ,ๆๆๆ,โปรแกรม,พัดลม",21,0.931051
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...,"เล่น,เกม,โน๊ตบุ๊ค,ดี,โลก,ราคา,28000,ทำได้,ประส...",20,0.968074
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก,"จุดเด่น,เสียง,ลำโพง,ดีมาก",25,0.871958
...,...,...,...,...
76,ขออนุญาตสอบถามหน่อยนะคะ คือเราซื้อ macbook pro...,"ขออนุญาต,สอบถาม,นะคะ,macbook,pro,14,”,m,1,pro,...",21,0.966275
77,อยากสอบถามทุกคนหน่อยค่ะ ว่า apple care+ ควรซื้...,"สอบถาม,apple,care,เครื่อง,icare,ต่างกัน, ,กลัว...",25,0.980692
78,ตั้งแต่เราซื้อผลิตภัณฑ์แอปเปิ้ลมาลังเลตลอดว่าจ...,"ผลิตภัณฑ์,แอปเปิ้ล,ลังเล,บ้าน,apple,ประกัน,ตอน...",16,0.986932
79,มันก็ออกมาเรื่อยทุกปีแหละครับ ดูการใช้งานและปร...,"ออกมา,ทุกปี,ดู,การใช้งาน,ประโยชน์,ที่จะ,คน,รอ,...",3,0.981247
